# CVIA Image Detection on SPARK dataset

`dataset.labels.iloc[1]`

Output:
```
class                            Terra
id                                6536
image              image_06536_img.png
depth            image_06536_depth.png
bbox              [476, 469, 542, 497]
illumination                  0.244364
rotation        [0.5, -0.6, -0.4, 0.5]
translation        [-3.0, -2.3, 147.9]
Name: 1, dtype: object
```

## Pytorch model preparation

Source: https://github.com/yhenon/pytorch-retinanet

**Use with:**

```bash
cd C:\Users\dvdme\OneDrive\Dokumente\cvia_course\cvia_semester_project\pytorch_retinanet
```

**Run without validation:**
    
```bash
python train.py --dataset csv --csv_train D:/Downloads/semester_project/project_files/pytorch_annotations_small.csv --csv_classes D:/Downloads/semester_project/project_files/classes.csv
```

**Or with validation:**

```bash
python train.py --dataset csv --csv_train D:/Downloads/semester_project/project_files/pytorch_annotations_small_train.csv --csv_classes D:/Downloads/semester_project/project_files/classes.csv --csv_val D:/Downloads/semester_project/project_files/pytorch_annotations_small_val.csv
```

**Run valitation:**

```bash
python csv_validation.py --csv_annotations_path D:/Downloads/semester_project/project_files/pytorch_annotations_small_val.csv --model_path csv_retinanet_90.pt --images_path D:/Downloads/semester_project/project_files/csv_annotations_small_val.csv --class_list_path D:/Downloads/semester_project/project_files/classes.csv  --iou_threshold 0.5
```

## Visualisation

```bash
python visualize.py --dataset csv --csv_classes D:/Downloads/semester_project/project_files/classes.csv --csv_val D:/Downloads/semester_project/project_files/pytorch_annotations_small_val.csv --model csv_retinanet_30.pt
```

Error appearing when launching anythin:

```
UserWarning: Failed to load image Python extension: Could not find module 'D:\Programs\Anaconda\envs\pytorch_retinanet\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
```

Error when `batch_size` is larger than 1:
```
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 8.46 MiB for an array with shape (608, 608, 3) and data type float64
```

In [1]:
from utils import SPARKDataset , PyTorchSparkDataset
from matplotlib import pyplot as plt
from random import randint

import os
import re
import zipfile

import numpy as np
import torch

import pandas as pd
from skimage import io , img_as_uint
import matplotlib.patches as mpatches

Found Pytorch


D:\Programs\Anaconda\envs\pytorch_retinanet\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\Programs\Anaconda\envs\pytorch_retinanet\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
print(f"numpy version: {np.__version__}")
print(f"pytorch version: {torch.__version__}")
print(f"pytorch version: {torch.version.cuda}")
torch.cuda.is_available()

numpy version: 1.22.1
pytorch version: 1.10.1+cu102
pytorch version: 10.2


True

In [5]:
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce GTX 1060


### Load dataset with utils.py

In [15]:
# set up the path to a local copy of the SPARK dataset, labels csv files should be in the same directory.

dataset_root_dir = 'D:/Downloads/semester_project/project_files/train_small/'

split = 'rgb'
test_length = 30000
dataset = SPARKDataset(root_dir=dataset_root_dir,split=split)


# class,id,image,depth,bbox,illumination,rotation,translation
labels_class = []
labels_id = []
labels_image = []
image_path = []
labels_depth = []
labels_bbox_xmin = []
labels_bbox_ymin = []
labels_bbox_xmax = []
labels_bbox_ymax = []


labels_bbox = []

labels_illumination = []


for i in range(test_length):
    labels_class.append(dataset.labels.iloc[i]['class'])
    labels_id.append(dataset.labels.iloc[i]['id'])
    labels_image.append(dataset.labels.iloc[i]['image'])
    labels_depth.append(dataset.labels.iloc[i]['depth'])
    labels_illumination.append(dataset.labels.iloc[i]['illumination'])
    
    labels_bbox_xmin.append(dataset.get_bbox(i)[0])
    labels_bbox_ymin.append(dataset.get_bbox(i)[1])
    labels_bbox_xmax.append(dataset.get_bbox(i)[2])
    labels_bbox_ymax.append(dataset.get_bbox(i)[3])
    
    labels_bbox.append([[dataset.get_bbox(i)[0], 
                        dataset.get_bbox(i)[1], 
                        dataset.get_bbox(i)[2], 
                        dataset.get_bbox(i)[3]],])
    image_path.append(os.path.join(dataset_root_dir, f"rgb/{label_dataframe['class'][i]}/{label_dataframe['image'][i]}"))

    
# data = {'class': labels_class,
#         'id': labels_id,
#         'image': labels_image,
#         'depth': labels_depth,
#         'illumination': labels_illumination,
#        }

# bbox_data = {'xmin': labels_bbox_xmin,
#              'ymin': labels_bbox_ymin,
#              'xmax': labels_bbox_xmax,
#              'ymax': labels_bbox_ymax,
#             }

data = {'image_path': image_path,
        'xmin': labels_bbox_xmin,
        'ymin': labels_bbox_ymin,
        'xmax': labels_bbox_xmax,
        'ymax': labels_bbox_ymax,
        'class': labels_class,
       }

my_dataframe = pd.DataFrame(data)

In [16]:
print(f'len(my_dataframe): {len(my_dataframe)}')
print()
my_dataframe.head()

len(my_dataframe): 30000



,image,xmin,ymin,xmax,ymax,class
0,D:/Downloads/semester_project/project_files/tr...,262,283,323,367,Cloudsat
1,D:/Downloads/semester_project/project_files/tr...,476,469,542,497,Terra
2,D:/Downloads/semester_project/project_files/tr...,520,445,584,516,Sentinel-6
3,D:/Downloads/semester_project/project_files/tr...,377,606,692,945,Sentinel-6
4,D:/Downloads/semester_project/project_files/tr...,523,421,560,464,CubeSat


In [17]:
my_dataframe.to_csv('D:/Downloads/semester_project/project_files/pytorch_annotations.csv', index=False, sep=',')

In [21]:
classes = ['AcrimSat',
           'Aquarius', 
           'Aura',
           'Calipso', 
           'Cloudsat', 
           'CubeSat',
           'Debris', 
           'Jason', 
           'Sentinel-6', 
           'Terra',
           'TRMM']
ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

class_mapping = {'class': classes,
                 'id': ids,
                }

class_frame = pd.DataFrame(class_mapping)
class_frame.to_csv('D:/Downloads/semester_project/project_files/classes.csv', index=False, sep=',')
class_frame

,class,id
0,AcrimSat,0
1,Aquarius,1
2,Aura,2
3,Calipso,3
4,Cloudsat,4
5,CubeSat,5
6,Debris,6
7,Jason,7
8,Sentinel-6,8
9,Terra,9


### Dataframe slicing

In [17]:
def visualize(i, label_dataframe, bbox_dataframe, root_dir, size=(15,15),  ax=None, image_type='rgb'):

        """ Visualizing image, with ground truth pose with axes projected to training image. """

        if ax is None:
            ax = plt.gca()
            
        image_path = os.path.join(root_dir, f"rgb/{label_dataframe['class'][i]}/{label_dataframe['image'][i]}")
        depth_path = os.path.join(root_dir, f"depth/{label_dataframe['class'][i]}/{label_dataframe['depth'][i]}")
        
        min_x = bbox_dataframe[0][i][0]
        min_y = bbox_dataframe[0][i][1]
        max_x = bbox_dataframe[0][i][2]
        max_y = bbox_dataframe[0][i][3]
        
        image = io.imread(image_path)
        depth = io.imread(depth_path)

        if image_type=='rgb':
            ax.imshow(image,vmin=0, vmax=255)
            my_color = 'red'
            im_scale = 1
        elif image_type=='depth':
            ax.imshow(depth, vmin=0, vmax=255)
            my_color = 'red'
            im_scale = 0.25
        else:
            print('Incorrect parameter: image_type. Exiting function!')
            return
        

        rect = mpatches.Rectangle((im_scale*min_y, im_scale*min_x), im_scale*(max_y - min_y), im_scale*(max_x - min_x),
                                    fill=False, edgecolor=my_color, linewidth=2)

        ax.add_patch(rect)
        
        #label = f"{list(self.class_map.keys())[list(self.class_map.values()).index(img_class)]}"
        
        #ax.text(min_y, min_x-20, label,color='white',fontsize=15)
        ax.set_axis_off()

        return 
root_dir = 'D:/Downloads/semester_project/project_files/train_small/'

## Create model conform dataset

#### Generate a dataframe slice with the images

In [14]:
# def generate_images_frame(label_dataframe, root_dir):
#     images = []
#     image_names = []
#     for i in range(len(label_dataframe['image'])):
#             image_path = os.path.join(root_dir, f"rgb/{label_dataframe['class'][i]}/{label_dataframe['image'][i]}")
# #             images.append(io.imread(image_path))
#             image_names.append(image_path)#("{label_dataframe['class'][i]}/{label_dataframe['image'][i]}")
#     return images, image_names

# dataset_images, image_names = generate_images_frame(label_dataframe, root_dir)

# def convert_ids(id_frame):
#     converted_ids = []
#     for element in id_frame:
#       converted_ids.append(np.array([element]))
#     return np.array(converted_ids)

#### Generate the tensorflow dataframe with the respective fields

In [78]:
# for i in range(len(label_dataframe)):
#     is_crowds.append(np.array(False))
# #np.zeros(len(label_dataframe))

# class_map =  {'AcrimSat':0, 'Aquarius':1, 'Aura':2, 'Calipso':3, 'Cloudsat':4, 'CubeSat':5,
#                            'Debris' :6, 'Jason':7, 'Sentinel-6':8, 'Terra':9, 'TRMM':10}
# remapped_labels = []
# for i in range(len(label_dataframe)):
#     remapped_labels.append(class_map[label_dataframe['class'][i]])
# remapped_labels = np.int64(remapped_labels)

# # Cast bbox to float
# labels_bbox = np.array(labels_bbox)
# labels_bbox = labels_bbox.astype(float)
# labels_bbox = np.float32(labels_bbox)
# labels_bbox = labels_bbox/1024 # may be wrong
# # labels_bbox = np.linalg.norm(labels_bbox)

# ## Remap to have an array of lists so that the variables can be read correctly with the indices
# converted_ids = convert_ids(label_dataframe['id'])
# remapped_labels = convert_ids(remapped_labels)

# dataset_objects = {'bbox': labels_bbox,
#                    'id': converted_ids,
#                    'label': remapped_labels,
#                   }

# tf_input_data = {'image': image_names, 
#                  'image/filename': label_dataframe['image'], 
#                  'image/id': label_dataframe['id'],
#                  'objects': dataset_objects
#                  }

In [28]:
model_dir = 'D:/Downloads/semester_project/project_files/pytorch_model/'

##  LOAD AND APPLY MODEL